# Generacion de jsons e insercion de estos en la base de datos

#### Una vez que tenemos los datos limpios, vamos a generar nuestros documentos (jsons) para nuestra base de datos no relacional (en este caso MongoDB) y posteriormente, dejarlos guardados en esta.

In [15]:
from pymongo.mongo_client import MongoClient
import pandas as pd
import json
import numpy as np
import os
from datetime import datetime, timedelta
import ast

In [16]:
# generamos nuestro diccionario con los dataframes de nuestras ofertas laborales
ruta = '../datos/datos_recibidos_empresa/datos_skills_identificadas/tratados/'
dic_dfs_ofertas = {archivo.rstrip('.csv') : pd.read_csv(f'{ruta}{archivo}') for archivo in os.listdir(ruta) if archivo.endswith('.csv')}

dic_dfs_ofertas.keys()

dict_keys(['ofertas_data_analyst_tratada', 'ofertas_data_engineer_tratada', 'ofertas_data_scientist_tratada', 'ofertas_seguridad_privacy_data_tratada'])

## Convertimos dataframes a jsons

In [17]:
if not os.listdir('../datos/datos_recibidos_empresa/datos_skills_identificadas/jsons/'):

    # Función para convertir columnas con listas en formato string a listas reales
    def convertir_columnas(df, columnas):
        for col in columnas:
            if col in df.columns:
                df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else [x])
        return df

    # Generar lista de fechas aleatorias
    fecha_inicio = datetime(2024, 8, 1)
    fecha_fin = datetime(2025, 2, 8)
    rango_fechas = [fecha_inicio + timedelta(days=i) for i in range((fecha_fin - fecha_inicio).days + 1)]

    for key, value in dic_dfs_ofertas.items():
        data = value

        # Asignar fechas aleatorias
        data['fecha'] = np.random.choice(rango_fechas, size=len(data)).astype(str)
        data['cat_identificada'] = 'Ninguna'

        # Convertir las columnas de str a list
        columnas_a_convertir = ["Salario", "Estudios minimos", "Skills"]
        data = convertir_columnas(data, columnas_a_convertir)

        # Convertir DataFrame a JSON por cada fila con el parametro 'orient'
        json_data = data.to_dict(orient='records')

        # Guardar en archivo JSON
        nombre_archivo = key.replace('_tratada', '')
        ruta_output = f'../datos/datos_recibidos_empresa/datos_skills_identificadas/jsons/{nombre_archivo}.json'
        with open(ruta_output, 'w') as json_file:
            json.dump(json_data, json_file, indent=4, ensure_ascii=False)

## Conexion a Mongo Atlas

In [ ]:

uri = "mongodb+srv://root:<db_password>@cluster0.pse9r.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)